In [8]:
import pickle
import os
import torch
import torch.nn.functional as F

In [9]:
results_path='/tsi/hi-paris/FCCLIP_results/All_results/INvocab/fcclip_cocopan_soft_label_cls_reslt'
image_id='frankfurt_000000_003025'

In [10]:
def read_data(results_path, image_id, iter):
    path_cls=os.path.join(results_path, "Softlabels", "iter_"+str(iter), "mask_cls_"+image_id)
    path_pred=os.path.join(results_path, "Softlabels", "iter_"+str(iter), "mask_pred_"+image_id)
    path_cls_results=os.path.join(results_path, "Softlabels", "iter_"+str(iter), "cls_results_"+image_id) 
    with open(path_cls, 'rb') as file:
        mask_cls_results=pickle.load(file)
    with open(path_pred, 'rb') as file:
        mask_pred_results=pickle.load(file)
    with open(path_cls_results, 'rb') as file:
        cls_results=pickle.load(file)
    return mask_cls_results, mask_pred_results, cls_results

In [11]:
mask_cls, mask_pred, cls_results=read_data(results_path,image_id, 0)

In [14]:
print("mask_cls shape : ",mask_cls.shape)
print("mask_pred shape : ",mask_pred.shape)
print("cls_results shape : ",cls_results.shape)

mask_cls shape :  torch.Size([250, 20])
mask_pred shape :  torch.Size([250, 1024, 2048])
cls_results shape :  torch.Size([250, 19])


In [13]:
cls_results

tensor([[ -4.5868,  -6.1712,  -5.3217,  ...,  -6.6361,  -5.7529,  -6.2488],
        [ -3.1796,  -0.5827,  -7.1274,  ...,  -7.4282,  -8.3347,  -7.3614],
        [ -4.6267,  -3.4722,  -4.5846,  ...,  -6.4773,  -2.5643,  -1.1388],
        ...,
        [ -4.7889,  -0.2774,  -7.3556,  ..., -10.5080, -10.2403,  -8.1663],
        [ -6.3979,  -7.4855,  -4.6816,  ...,  -6.1244,  -7.6204,  -6.7934],
        [ -7.1083,  -7.3084,  -0.5636,  ...,  -6.0019,  -8.5052,  -6.5708]])

In [16]:
mask_cls

tensor([[-8.1133e+00, -9.6976e+00, -8.8482e+00,  ..., -9.2793e+00,
         -9.7752e+00, -2.5161e-02],
        [-3.7035e+00, -1.1066e+00, -7.6513e+00,  ..., -8.8586e+00,
         -7.8853e+00, -4.7646e-01],
        [-8.5122e+00, -7.3577e+00, -8.4701e+00,  ..., -6.4498e+00,
         -5.0243e+00, -2.0750e-02],
        ...,
        [-6.7635e+00, -2.2520e+00, -9.3300e+00,  ..., -1.2213e+01,
         -1.0141e+01, -1.4946e-01],
        [-9.3261e+00, -1.0413e+01, -7.6098e+00,  ..., -1.0548e+01,
         -9.7215e+00, -5.1666e-02],
        [-1.2587e+01, -1.2786e+01, -6.0452e+00,  ..., -1.3975e+01,
         -1.2051e+01, -2.8985e-03]])

In [15]:
mask_pred

tensor([[[-40.6938, -40.6938, -41.2296,  ..., -49.4048, -50.0707, -50.0707],
         [-40.6938, -40.6938, -41.2296,  ..., -49.4048, -50.0707, -50.0707],
         [-41.3912, -41.3912, -41.8377,  ..., -49.0957, -49.7322, -49.7322],
         ...,
         [-35.2304, -35.2304, -34.6074,  ..., -26.0899, -26.1625, -26.1625],
         [-35.6348, -35.6348, -34.9745,  ..., -26.5478, -26.6025, -26.6025],
         [-35.6348, -35.6348, -34.9745,  ..., -26.5478, -26.6025, -26.6025]],

        [[-12.5402, -12.5402, -12.6221,  ..., -11.9848, -11.9099, -11.9099],
         [-12.5402, -12.5402, -12.6221,  ..., -11.9848, -11.9099, -11.9099],
         [-12.6390, -12.6390, -12.6980,  ..., -11.8489, -11.7799, -11.7799],
         ...,
         [ -8.4243,  -8.4243,  -7.9612,  ...,  -4.9468,  -5.0400,  -5.0400],
         [ -8.7508,  -8.7508,  -8.2640,  ...,  -5.0188,  -5.1018,  -5.1018],
         [ -8.7508,  -8.7508,  -8.2640,  ...,  -5.0188,  -5.1018,  -5.1018]],

        [[-70.2140, -70.2140, -71.5643,  ...

In [5]:
def get_results(mask_cls, mask_pred): 
    mask_pred_view = mask_pred.view(250, -1)   
  
    result = torch.matmul(mask_pred_view.T,mask_cls)
 
    result_reshaped = result.view(19, 1024, 2048)
 
    softmax_result = F.softmax(result_reshaped.T, dim=-1)
    return softmax_result


In [6]:
results=get_results(mask_cls, mask_pred)

/tmp/ipykernel_4136620/1249224819.py:8: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3675.)
  softmax_result = F.softmax(result_reshaped.T, dim=-1)


In [7]:
def get_possibility_of_erreur(results, seuil):
 
    greater_than_seuil = results > seuil

    count_greater_than_suil = greater_than_seuil.sum(dim=-1)
 
    conflict_mask = count_greater_than_suil > 1
 
    sum_of_conflict = conflict_mask.sum().item()

    print("Number of conflicts:", sum_of_conflict)


In [8]:
get_possibility_of_erreur(results, 0.4)

Number of conflicts: 145


# approach  1 
### The criterions
To detect cases were probabilities of labels are similar

Case were there is a risk of change in final label prediction

We are looking at a pixel level per image an array of 19 labels with probabilities

Examples: Standard Deviation between values 

In [16]:
def get_possibility_of_erreur_std(results):
 
    std_deviation = results.std(dim=-1)
    
    conflict_mask = std_deviation > 1
 
    sum_of_conflict = conflict_mask.sum().item()

    print("Number of conflicts:", sum_of_conflict)

In [17]:
get_possibility_of_erreur_std(results)

tensor([[0.2294, 0.2294, 0.2294,  ..., 0.2294, 0.2294, 0.2294],
        [0.2294, 0.2294, 0.2294,  ..., 0.2294, 0.2294, 0.2294],
        [0.2294, 0.2294, 0.2294,  ..., 0.2294, 0.2294, 0.2294],
        ...,
        [0.2294, 0.2294, 0.2294,  ..., 0.2294, 0.2294, 0.2294],
        [0.2294, 0.2294, 0.2294,  ..., 0.2294, 0.2294, 0.2294],
        [0.2294, 0.2294, 0.2294,  ..., 0.2294, 0.2294, 0.2294]])
Number of conflicts: 0
